In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3" # Optional set GPU device ID
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [2]:
from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 04-14 07:26:09 [__init__.py:239] Automatically detected platform cuda.


In [3]:
model_name = "/home/tianhaoyu/.cache/modelscope/hub/models/LLM-Research/Llama-3.2-3B-Instruct"

In [ ]:
from unsloth import is_bfloat16_supported
import torch
max_seq_length = 4096*2 # Can increase for longer reasoning traces
lora_rank = 64 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.2, # Reduce if out of memory
)

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.1. vLLM: 0.8.2.
   \\   /|    NVIDIA GeForce RTX 4090 D. Num GPUs = 4. Max memory: 47.382 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading /home/tianhaoyu/.cache/modelscope/hub/models/LLM-Research/Llama-3.2-3B-Instruct with actual GPU utilization = 59.51%
Unsloth: Your GPU has CUDA compute capability 8.9 with VRAM = 47.38 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 8192. Num Sequences = 288.
Unsloth: vLLM's KV Cache can use up to 22.03 GB. Also swap space = 6 GB.
INFO 04-14 07:28:20 [config.py:585] This model supports multiple tasks: {'score', 'generate', 'embed', 'reward', 'classify'}. Defaulting to

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 04-14 07:28:22 [punica_selector.py:18] Using PunicaWrapperGPU.
INFO 04-14 07:28:22 [model_runner.py:1146] Model loading took 2.3520 GB and 0.897224 seconds
INFO 04-14 07:28:26 [worker.py:267] Memory profiling takes 3.03 seconds
INFO 04-14 07:28:26 [worker.py:267] the current vLLM instance can use total_gpu_memory (47.38GiB) x gpu_memory_utilization (0.60) = 28.20GiB
INFO 04-14 07:28:26 [worker.py:267] model weights take 2.35GiB; non_torch_memory takes 0.07GiB; PyTorch activation peak memory takes 1.38GiB; the rest of the memory reserved for KV Cache is 24.40GiB.
INFO 04-14 07:28:26 [executor_base.py:111] # cuda blocks: 14276, # CPU blocks: 3510
INFO 04-14 07:28:26 [executor_base.py:116] Maximum concurrency for 8192 tokens per request: 27.88x
INFO 04-14 07:28:28 [model_runner.py:1442] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If

Capturing CUDA graph shapes: 100%|██████████| 39/39 [00:25<00:00,  1.52it/s]

INFO 04-14 07:28:53 [model_runner.py:1570] Graph capturing finished in 26 secs, took 0.68 GiB
INFO 04-14 07:28:53 [llm_engine.py:447] init engine (profile, create kv cache, warmup model) took 30.89 seconds


/home/tianhaoyu/.cache/modelscope/hub/models/LLM-Research/Llama-3.2-3B-Instruct does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.


In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

Unsloth 2025.3.19 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [6]:
import torch
print(f"Number of available GPUs: {torch.cuda.device_count()}")

Number of available GPUs: 4


In [7]:
import re
from datasets import load_dataset, Dataset
from search_module import search, get_question_answer, get_question_count
from rl_helpers import get_qa_dataset

train_dataset, test_dataset = get_qa_dataset()

Loading FAISS index from: /home/tianhaoyu/AutoDidact/faiss_index
Successfully loaded FAISS index
Loading chunks from: /home/tianhaoyu/AutoDidact/saved_data/chunks.pkl
Loading questions from: /home/tianhaoyu/AutoDidact/saved_data/questions.json
Successfully loaded 341 chunks and 678 questions


<a name="Train"></a>
### Train the model

Now set up GRPO Trainer and all configurations!

In [8]:
import os
os.environ["WANDB_PROJECT"] = "bootstrap-search-rl"
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3,4"

In [9]:
# from UnslothGRPOTrainerTemp import UnslothGRPOConfig, _UnslothGRPOTrainer
import UnslothGRPOTrainerTemp
training_args = UnslothGRPOTrainerTemp.UnslothGRPOConfig(
    use_vllm = True, # use vLLM for fast inference!
    use_agentic_generate = True, # use agentic generation
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "paged_adamw_8bit",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 8,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 8, # Decrease if out of memory
    max_prompt_length = 1024,
    max_completion_length = 1024,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 101,
    save_steps = 50,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "full_local_training",

)

In [10]:


import rl_helpers
# importlib.reload(rl_helpers)


def agentic_generate(
        prompts:list[str],
        generate_fn,
        max_generations:int=6,
        ):
    return run_agent(generate_fn, tokenizer, prompts, max_generations)
model.agentic_generate = agentic_generate


from vllm import SamplingParams
verifier_sampling_params = SamplingParams(
    temperature = 0.1,
    top_p = 0.95,
    max_tokens = 4096,
)
def verifier_generate_fn(inputs):
    return model.fast_generate(
        inputs,
        sampling_params = verifier_sampling_params,
    )


run_agent = rl_helpers.run_agent
reward_correctness = rl_helpers.build_reward_correctness_fn(verifier_generate_fn, tokenizer,)
reward_formatting = rl_helpers.reward_formatting

import UnslothGRPOTrainerTemp
trainer = UnslothGRPOTrainerTemp.UnslothGRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        reward_correctness,
        reward_formatting,
    ],
    args = training_args,
    train_dataset = train_dataset,
)

In [11]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 610 | Num Epochs = 1 | Total steps = 101
O^O/ \_/ \    Batch size per device = 32 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (32 x 1 x 1) = 32
 "-____-"     Trainable parameters = 97,255,424/3,000,000,000 (3.24% trained)


['What was the achieved pericynthion altitude at translunar injection?', 'What was the achieved pericynthion altitude at translunar injection?', 'What was the achieved pericynthion altitude at translunar injection?', 'What was the achieved pericynthion altitude at translunar injection?', 'What was the achieved pericynthion altitude at translunar injection?', 'What was the achieved pericynthion altitude at translunar injection?', 'What was the achieved pericynthion altitude at translunar injection?', 'What was the achieved pericynthion altitude at translunar injection?', 'What was the probable cause of the large negative field of approximately minus 3000 volts/meter at site 6 after the oscillations had subsided at T plus 40 seconds?', 'What was the probable cause of the large negative field of approximately minus 3000 volts/meter at site 6 after the oscillations had subsided at T plus 40 seconds?', 'What was the probable cause of the large negative field of approximately minus 3000 volt

/home/tianhaoyu/AutoDidact/UnslothGRPOTrainerTemp.py:943: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  completion_ids = [torch.tensor(ids, device=device) for ids in completion_ids]
Processed prompts: 100%|██████████| 32/32 [00:00<00:00, 155.52it/s, est. speed input: 19961.42 toks/s, output: 311.13 toks/s]


rewards_per_func: tensor([0., 0.], device='cuda:0')


Step,Training Loss,reward,reward_std,completion_length,kl,rewards / reward_correctness,rewards / reward_formatting
1,0.000000,0.000000,0.000000,126.875000,0.000000,0.000000,0.000000
2,-0.000000,0.118750,0.216235,157.906250,0.000000,0.031250,0.087500
3,0.000000,0.106250,0.300520,84.312500,0.000339,0.062500,0.043750
4,0.000000,0.053125,0.150260,101.468750,0.000418,0.031250,0.021875
5,0.000000,0.150000,0.366242,134.343750,0.000565,0.062500,0.087500
6,0.000000,0.128125,0.362392,124.000000,0.000463,0.062500,0.065625
7,0.000000,0.075000,0.212132,155.750000,0.000486,0.031250,0.043750
8,0.000000,0.106250,0.196737,104.593750,0.000581,0.062500,0.043750
9,0.000000,0.075000,0.212132,109.250000,0.000508,0.031250,0.043750
10,0.000000,0.140625,0.312279,134.437500,0.001091,0.031250,0.109375


...

Processed prompts: 100%|██████████| 32/32 [00:00<00:00, 125.39it/s, est. speed input: 26098.29 toks/s, output: 266.51 toks/s]


rewards_per_func: tensor([0.4688, 0.6781], device='cuda:0')
['What was the cause of the cracking in the window shades on the Apollo 13 mission?', 'What was the cause of the cracking in the window shades on the Apollo 13 mission?', 'What was the cause of the cracking in the window shades on the Apollo 13 mission?', 'What was the cause of the cracking in the window shades on the Apollo 13 mission?', 'What was the cause of the cracking in the window shades on the Apollo 13 mission?', 'What was the cause of the cracking in the window shades on the Apollo 13 mission?', 'What was the cause of the cracking in the window shades on the Apollo 13 mission?', 'What was the cause of the cracking in the window shades on the Apollo 13 mission?', 'What was the heart rate of the Lunar Module Pilot during the entry phase?', 'What was the heart rate of the Lunar Module Pilot during the entry phase?', 'What was the heart rate of the Lunar Module Pilot during the entry phase?', 'What was the heart rate of 

/home/tianhaoyu/AutoDidact/UnslothGRPOTrainerTemp.py:943: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  completion_ids = [torch.tensor(ids, device=device) for ids in completion_ids]
Processed prompts: 100%|██████████| 32/32 [00:00<00:00, 149.59it/s, est. speed input: 27585.52 toks/s, output: 299.23 toks/s]


rewards_per_func: tensor([0.8125, 0.6781], device='cuda:0')


TrainOutput(global_step=101, training_loss=0.0008406649752518729, metrics={'train_runtime': 2455.0322, 'train_samples_per_second': 1.316, 'train_steps_per_second': 0.041, 'total_flos': 0.0, 'train_loss': 0.0008406649752518729})

<a name="Inference"></a>
### Inference
Now let's try benchmark the model we trained!

In [14]:
from vllm import SamplingParams
import rl_helpers
sampling_params = SamplingParams(
    temperature = 0.5,
    top_p = 0.95,
    max_tokens = 4096,
)

def eval_generate_fn(inputs):
    return model.fast_generate(
        inputs,
        sampling_params = sampling_params,
        lora_request = model.load_lora("full_local_training/checkpoint-101"), # load the trained LoRA
    )


rl_helpers.run_eval(
    generate_fn=eval_generate_fn,
    verify_fn=reward_correctness,
    tokenizer=tokenizer,
)

Processed prompts:   0%|          | 0/68 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts: 100%|██████████| 68/68 [00:00<00:00, 269.55it/s, est. speed input: 47334.35 toks/s, output: 543.19 toks/s]

RESULTS:
percentage of correct answers: 0.6176470588235294


[{'messages': [{'role': 'system',
    'content': 'Cutting Knowledge Date: December 2023\nToday Date: 14 Apr 2025\n\nWhen you receive a tool call response, use the output to format an answer to the original user question.\n\nYou are a helpful assistant with tool calling capabilities.\n'},
   {'role': 'user',
    'content': 'You are a research assistant, and you use the search_corpus tool to find answers to questions.\nGiven a question, answer it using by doing searches using the search_corpus tool.\nTo use the search_corpus tool, respond with a JSON for a function call with its proper arguments.\n\nYou may also reason in any message, thinking step by step about how to answer the question. Wrap your reasoning in <reasoning> and </reasoning> tags.\n\n{\n  "type": "function",\n  "function": {\n    "name": "search_corpus",\n    "description": "Search over the knowledge corpus with a given query",\n    "parameters": {\n      "type": "object",\n      "properties": {\n        "query": {\n     

In [15]:
# eval w/o lora
def eval_generate_fn(inputs):
    return model.fast_generate(
        inputs,
        sampling_params = sampling_params,
    )


rl_helpers.run_eval(
    generate_fn=eval_generate_fn,
    verify_fn=reward_correctness,
    tokenizer=tokenizer,
)

Processed prompts:   0%|          | 0/68 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts: 100%|██████████| 68/68 [00:00<00:00, 631.43it/s, est. speed input: 80992.35 toks/s, output: 1263.43 toks/s]


RESULTS:
percentage of correct answers: 0.0


[{'messages': [{'role': 'system',
    'content': 'Cutting Knowledge Date: December 2023\nToday Date: 14 Apr 2025\n\nWhen you receive a tool call response, use the output to format an answer to the original user question.\n\nYou are a helpful assistant with tool calling capabilities.\n'},
   {'role': 'user',
    'content': 'You are a research assistant, and you use the search_corpus tool to find answers to questions.\nGiven a question, answer it using by doing searches using the search_corpus tool.\nTo use the search_corpus tool, respond with a JSON for a function call with its proper arguments.\n\nYou may also reason in any message, thinking step by step about how to answer the question. Wrap your reasoning in <reasoning> and </reasoning> tags.\n\n{\n  "type": "function",\n  "function": {\n    "name": "search_corpus",\n    "description": "Search over the knowledge corpus with a given query",\n    "parameters": {\n      "type": "object",\n      "properties": {\n        "query": {\n     